In [33]:
import os
import pandas as pd
import numpy as np
from textblob import TextBlob


In [34]:
os.chdir('../scripts/')
import data_loader as data_loader

In [35]:
all_data_path = '../data/raw_analyst_ratings.csv'
df = data_loader.read_csv_file(all_data_path)

In [36]:
df = df.get("data")
df.head()

,headline,url,publisher,date,stock
0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A


In [28]:
df['date'] = pd.to_datetime(df['date'], format='ISO8601')
df.head()

,headline,url,publisher,date,stock
0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A


In [40]:
all_data_path = '../data/yfinance_data/AAPL_historical_data.csv'
df2 = data_loader.read_csv_file(all_data_path)
df2 = df2.get("data")
df2['Date'] = pd.to_datetime(df2['Date'], format='ISO8601')

df2.head()
df2.shape

(10998, 9)

In [41]:
# Create a DataFrame with dates from the AAPL data
dates = df2.index
text_data = pd.DataFrame(dates, columns=['Date'])


In [42]:
# Assign a random headline to each date
text_data['Headline'] = np.random.choice(df['headline'], size=len(text_data))

# Display the first few rows of the updated DataFrame
text_data.head(30)

,Date,Headline
0,0,Wunderlich Initiates Coverage on Whitewave Foo...
1,1,Bloomberg Reporting Uni-Pixel Has Said Preferr...
2,2,German Parliament Unlikely To Back Major Loan ...
3,3,Lenovo Spokesperson Declines Comment on Chatte...
4,4,13 Stocks Jim Cramer Is Looking At
5,5,Mid-Morning Market Update: Markets Gain; Apple...
6,6,Stocks That Hit 52-Week Highs On Monday
7,7,MTBC Sees FY20 Sales $100M-$102M
8,8,Benzinga's Top Upgrades
9,9,"Stocks Which Set New 52-Week High Yesterday, S..."


In [43]:
def calculate_sentiment(text):
    return TextBlob(text).sentiment.polarity

# Applying the sentiment analysis function to your headlines
text_data['Sentiment'] = text_data['Headline'].apply(calculate_sentiment)

In [46]:
combined_data = text_data.set_index('Date').join(df2['Close'])
combined_data.head()
combined_data['Daily Returns'] = combined_data['Close'].pct_change()
combined_data.head()

,Headline,Sentiment,Close,Daily Returns
Date,,,,
0,Wunderlich Initiates Coverage on Whitewave Foo...,0.0000,0.128348,NaN
1,Bloomberg Reporting Uni-Pixel Has Said Preferr...,0.0000,0.121652,-0.052171
2,German Parliament Unlikely To Back Major Loan ...,-0.0875,0.112723,-0.073398
3,Lenovo Spokesperson Declines Comment on Chatte...,0.0000,0.115513,0.024751
4,13 Stocks Jim Cramer Is Looking At,0.0000,0.118862,0.028992


In [47]:
correlation = combined_data['Sentiment'].corr(combined_data['Daily Returns'])
print("Correlation between sentiment and daily stock returns:", correlation)


Correlation between sentiment and daily stock returns: 0.005436440193270533


In [48]:
correlation = combined_data['Sentiment'].corr(combined_data['Close'])
print("Correlation between sentiment and closing:", correlation)

Correlation between sentiment and closing: 0.008883134368783544
